# SVMによるボクセルごとの学習と性能評価（時系列解析）
----

引数：raw_rock.csv/raw_scissor.csv/raw_paper.csvがあるディレクトリまでのパス

----

入力：raw_rock.csv/raw_scissor.csv/raw_paper.csv

----

出力：ACCURACY[loo or k_cv]_VOXtimeseries[1時系列あたりのスキャン数]_SVM.csv ボクセルごとの識別性能評価結果一覧

----
ボクセルごとにあるデータにおけるNスキャン分の時系列データをテストデータ，テストデータを除いた残りのデータにおけるNスキャン分の時系列データ1ずつずらしながらを学習データとして取得し，SVMを用いて学習，交差検証法を用いて識別性能評価を行う．  
ベクトル：各ボクセルにおけるデータにおけるNスキャン分の時系列データ

In [1]:
print('############ ML_SVM_VOXtimeseries.py program excution ############')

############ ML_SVM_VOXtimeseries.py program excution ############


In [2]:
import numpy as np
import pandas as pd
import sys

from sklearn import svm

In [3]:
#args = sys.argv
#PATH = args[1]

# jupyter notebookのときはここで指定
PATH = '../State-2fe_Active/20181029rn/64ch/RawData/'


# 検証手法
col_name = 'leave-one-out'

In [4]:
headcoil = PATH.split('/')[3]


if headcoil == 'mb':
    
    # 1時系列あたりスキャン数
    N = 30
    
else :

    # 1時系列あたりスキャン数
    N = 10

print("TimeSeries(Scan Num) : " + str(N))


TimeSeries(Scan Num) : 10


## TsShift関数
引数としてあるボクセルにおける全試行分のデータをdata，タスクを見分けるための番号をlabelに受け取る．  
各試行で1ずつずらしながらNスキャン分の時系列データを取得する．全試行の時系列データをまとめて返す．

In [9]:
def TsShift(data, label):
    
    # 1ボクセルあたりで取得する時系列データ格納用データフレーム
    ts_all = pd.DataFrame(index = [], columns = [])

    # 時系列として扱う区間の始まり
    ts_fst = 0

    # 時系列として扱う区間の終わり
    ts_end = N

    # 1ずつずらしながら時系列データを取得，結合
    while ts_end <= len(data):

        # 時系列として扱う区間の始まり，区間の終わり，その区間（Nスキャン分）の時系列データの順でリスト化
        ts = [label] + [ts_fst + 1] + [ts_end] + list(data[ts_fst:ts_end])

        # データフレーム化，結合
        ts = pd.DataFrame(ts).T

        ts_all = pd.concat([ts_all, ts])

        # 1ずつずらす
        ts_fst = ts_fst + 1
        ts_end = ts_end + 1
            
            
    return ts_all

## SVM_LOO関数
学習と評価に用いるデータをdataで受け取り．  
データからテストデータ，テストデータラベル，教師データ，教師データラベルを生成．  
テストデータはdataにおける全時系列データ，教師データは条件に当てはまる時系列データであり，  
テストデータに用いるデータごとに識別できたかできなかったか（1か0）を取得，全テストデータで識別できた(1)の割合を算出（leave-one-outと同じ要領）．  
得られた割合をパーセント表記にし，main関数へ返す．

In [13]:
def SVM_LOO(data):

    scores = np.zeros(len(data))

    for i in range(len(data)):
        
        print("--------" + str(i + 1) + " / " + str(len(data)) + "---------")

        # テストデータの情報（タスク，試行数，時系列データの始まり，終わり）を取得
        test_label = data.iloc[i, 0]
        test_fst = data.iloc[i, 1]
        test_end = data.iloc[i, 2]


        ###### テストデータ

        # 時系列データのみを抽出
        X_test = data.iloc[i, 3:len(data.columns)]

        # ベクトル化
        X_test = X_test.as_matrix()
        X_test = X_test.reshape(1, -1)

        # ラベルを作成
        y_test = np.array([test_label])
        y_test = y_test.reshape(1, -1)

        ###### 教師データ

        # テストデータではない，テストデータに含まれるZ-scoreを含まないものを取得
        traindata = data[(data['label'] != test_label) | (data['fst'] > test_end) | (data['end'] < test_fst)]

        # 時系列データのみを抽出
        X_train = traindata.iloc[:, 3:len(data.columns)]

        # ベクトル化
        X_train = X_train.as_matrix()

        # ラベルを作成
        y_train = np.array(list(traindata['label']))

        print("Test Data : " + str(test_fst) + "-" + str(test_end) + " / Train Data Num : " + str(len(traindata)))
                                                                                        
        # 線形SVMのインスタンスを生成
        model = svm.SVC(kernel = 'linear', C=1)

        # モデルの学習
        model.fit(X_train, y_train)

        # 評価結果を格納
        scores[i] = model.score(X_test, y_test)
        
    # 評価結果の平均を求める
    result = scores.mean()
              
    
    # パーセント表記へ
    result = round(result * 100, 1)
    
    print(str(scores) + '\n')
    
    return result


In [14]:
if __name__ == '__main__':
    
    # 読み込みたいファイルのパス
    PATH_rock = PATH + 'raw_rock.csv'
    PATH_scissor = PATH + 'raw_scissor.csv'
    PATH_paper = PATH + 'raw_paper.csv'
    
    
    # csvファイル読み込み
    # headerは設定せず，転置後にset_index()する（header = 0にすると列名が変えられる）
    rock = pd.read_csv(PATH_rock, header = None, index_col = None).T
    rock.columns = range(0, len(rock.columns))
    rock = rock.set_index(0)
    
    scissor = pd.read_csv(PATH_scissor, header = None, index_col = None).T
    scissor.columns = range(0, len(scissor.columns))
    scissor = scissor.set_index(0)
    
    paper = pd.read_csv(PATH_paper, header = None, index_col = None).T
    paper.columns = range(0, len(paper.columns))
    paper = paper.set_index(0)

In [15]:
    # 全ボクセルの識別率を格納するデータフレーム
    voxAc = pd.DataFrame(index = sorted(list(set(rock.index))), columns = [col_name])
    
    # ボクセル数
    voxNum = len(rock)
    
    for voxNo in range(voxNum):
    
        voxName = 'Voxel' + str(voxNo + 1)

        print(voxName)

        # ボクセルのデータを取得
        rockVox = rock.loc[voxName]
        scissorVox = scissor.loc[voxName]
        paperVox = paper.loc[voxName]
        
        # ボクセルにおける時系列データを取得
        rockVoxTs = TsShift(rockVox, 0)
        scissorVoxTs = TsShift(scissorVox, 1)
        paperVoxTs = TsShift(paperVox, 2)
        
        # 全タスクを縦結合
        VoxTs = pd.concat([rockVoxTs, scissorVoxTs, paperVoxTs])
        
        # 0-3列目は条件判定用の要素，要素名をつけておく
        col_names = list(VoxTs.columns)
        col_names[0:3] = ['label', 'fst', 'end']
        VoxTs.columns = col_names
        
        VoxTs.index = range(0,len(VoxTs))
        
        # 学習と評価
        result_vox = SVM_LOO(VoxTs)
        
        print(result_vox)
        
        # データフレームに格納
        voxAc.at[voxName, col_name] = result_vox
        

Voxel1
--------1 / 261---------
Test Data : 1-10 / Train Data Num : 251
--------2 / 261---------
Test Data : 2-11 / Train Data Num : 250
--------3 / 261---------
Test Data : 3-12 / Train Data Num : 249
--------4 / 261---------
Test Data : 4-13 / Train Data Num : 248
--------5 / 261---------
Test Data : 5-14 / Train Data Num : 247
--------6 / 261---------
Test Data : 6-15 / Train Data Num : 246
--------7 / 261---------
Test Data : 7-16 / Train Data Num : 245
--------8 / 261---------
Test Data : 8-17 / Train Data Num : 244
--------9 / 261---------
Test Data : 9-18 / Train Data Num : 243
--------10 / 261---------
Test Data : 10-19 / Train Data Num : 242
--------11 / 261---------
Test Data : 11-20 / Train Data Num : 242
--------12 / 261---------
Test Data : 12-21 / Train Data Num : 242
--------13 / 261---------
Test Data : 13-22 / Train Data Num : 242
--------14 / 261---------
Test Data : 14-23 / Train Data Num : 242
--------15 / 261---------
Test Data : 15-24 / Train Data Num : 242
------

Test Data : 59-68 / Train Data Num : 242
--------147 / 261---------
Test Data : 60-69 / Train Data Num : 242
--------148 / 261---------
Test Data : 61-70 / Train Data Num : 242
--------149 / 261---------
Test Data : 62-71 / Train Data Num : 242
--------150 / 261---------
Test Data : 63-72 / Train Data Num : 242
--------151 / 261---------
Test Data : 64-73 / Train Data Num : 242
--------152 / 261---------
Test Data : 65-74 / Train Data Num : 242
--------153 / 261---------
Test Data : 66-75 / Train Data Num : 242
--------154 / 261---------
Test Data : 67-76 / Train Data Num : 242
--------155 / 261---------
Test Data : 68-77 / Train Data Num : 242
--------156 / 261---------
Test Data : 69-78 / Train Data Num : 242
--------157 / 261---------
Test Data : 70-79 / Train Data Num : 242
--------158 / 261---------
Test Data : 71-80 / Train Data Num : 242
--------159 / 261---------
Test Data : 72-81 / Train Data Num : 242
--------160 / 261---------
Test Data : 73-82 / Train Data Num : 242
-------


0.8
Voxel2
--------1 / 261---------
Test Data : 1-10 / Train Data Num : 251
--------2 / 261---------
Test Data : 2-11 / Train Data Num : 250
--------3 / 261---------
Test Data : 3-12 / Train Data Num : 249
--------4 / 261---------
Test Data : 4-13 / Train Data Num : 248
--------5 / 261---------
Test Data : 5-14 / Train Data Num : 247
--------6 / 261---------
Test Data : 6-15 / Train Data Num : 246
--------7 / 261---------
Test Data : 7-16 / Train Data Num : 245
--------8 / 261---------
Test Data : 8-17 / Train Data Num : 244
--------9 / 261---------
Test Data : 9-18 / Train Data Num : 243
--------10 / 261---------
Test Data : 10-19 / Train Data Num : 242
--------11 / 261---------
Test Data : 11-20 / Train Data Num : 242
--------12 / 261---------
Test Data : 12-21 / Train Data Num : 242
--------13 / 261---------
Test Data : 13-22 / Train Data Num : 242
--------14 / 261---------
Test Data : 14-23 / Train Data Num : 242
--------15 / 261---------
Test Data : 15-24 / Train Data Num : 242
-

--------140 / 261---------
Test Data : 53-62 / Train Data Num : 242
--------141 / 261---------
Test Data : 54-63 / Train Data Num : 242
--------142 / 261---------
Test Data : 55-64 / Train Data Num : 242
--------143 / 261---------
Test Data : 56-65 / Train Data Num : 242
--------144 / 261---------
Test Data : 57-66 / Train Data Num : 242
--------145 / 261---------
Test Data : 58-67 / Train Data Num : 242
--------146 / 261---------
Test Data : 59-68 / Train Data Num : 242
--------147 / 261---------
Test Data : 60-69 / Train Data Num : 242
--------148 / 261---------
Test Data : 61-70 / Train Data Num : 242
--------149 / 261---------
Test Data : 62-71 / Train Data Num : 242
--------150 / 261---------
Test Data : 63-72 / Train Data Num : 242
--------151 / 261---------
Test Data : 64-73 / Train Data Num : 242
--------152 / 261---------
Test Data : 65-74 / Train Data Num : 242
--------153 / 261---------
Test Data : 66-75 / Train Data Num : 242
--------154 / 261---------
Test Data : 67-76 / T

Voxel3
--------1 / 261---------
Test Data : 1-10 / Train Data Num : 251
--------2 / 261---------
Test Data : 2-11 / Train Data Num : 250
--------3 / 261---------
Test Data : 3-12 / Train Data Num : 249
--------4 / 261---------
Test Data : 4-13 / Train Data Num : 248
--------5 / 261---------
Test Data : 5-14 / Train Data Num : 247
--------6 / 261---------
Test Data : 6-15 / Train Data Num : 246
--------7 / 261---------
Test Data : 7-16 / Train Data Num : 245
--------8 / 261---------
Test Data : 8-17 / Train Data Num : 244
--------9 / 261---------
Test Data : 9-18 / Train Data Num : 243
--------10 / 261---------
Test Data : 10-19 / Train Data Num : 242
--------11 / 261---------
Test Data : 11-20 / Train Data Num : 242
--------12 / 261---------
Test Data : 12-21 / Train Data Num : 242
--------13 / 261---------
Test Data : 13-22 / Train Data Num : 242
--------14 / 261---------
Test Data : 14-23 / Train Data Num : 242
--------15 / 261---------
Test Data : 15-24 / Train Data Num : 242
------

...

--------1 / 261---------
Test Data : 1-10 / Train Data Num : 251
--------2 / 261---------
Test Data : 2-11 / Train Data Num : 250
--------3 / 261---------
Test Data : 3-12 / Train Data Num : 249
--------4 / 261---------
Test Data : 4-13 / Train Data Num : 248
--------5 / 261---------
Test Data : 5-14 / Train Data Num : 247
--------6 / 261---------
Test Data : 6-15 / Train Data Num : 246
--------7 / 261---------
Test Data : 7-16 / Train Data Num : 245
--------8 / 261---------
Test Data : 8-17 / Train Data Num : 244
--------9 / 261---------
Test Data : 9-18 / Train Data Num : 243
--------10 / 261---------
Test Data : 10-19 / Train Data Num : 242
--------11 / 261---------
Test Data : 11-20 / Train Data Num : 242
--------12 / 261---------
Test Data : 12-21 / Train Data Num : 242
--------13 / 261---------
Test Data : 13-22 / Train Data Num : 242
--------14 / 261---------
Test Data : 14-23 / Train Data Num : 242
--------15 / 261---------
Test Data : 15-24 / Train Data Num : 242
--------16 / 

Test Data : 43-52 / Train Data Num : 242
--------131 / 261---------
Test Data : 44-53 / Train Data Num : 242
--------132 / 261---------
Test Data : 45-54 / Train Data Num : 242
--------133 / 261---------
Test Data : 46-55 / Train Data Num : 242
--------134 / 261---------
Test Data : 47-56 / Train Data Num : 242
--------135 / 261---------
Test Data : 48-57 / Train Data Num : 242
--------136 / 261---------
Test Data : 49-58 / Train Data Num : 242
--------137 / 261---------
Test Data : 50-59 / Train Data Num : 242
--------138 / 261---------
Test Data : 51-60 / Train Data Num : 242
--------139 / 261---------
Test Data : 52-61 / Train Data Num : 242
--------140 / 261---------
Test Data : 53-62 / Train Data Num : 242
--------141 / 261---------
Test Data : 54-63 / Train Data Num : 242
--------142 / 261---------
Test Data : 55-64 / Train Data Num : 242
--------143 / 261---------
Test Data : 56-65 / Train Data Num : 242
--------144 / 261---------
Test Data : 57-66 / Train Data Num : 242
-------

Voxel7
--------1 / 261---------
Test Data : 1-10 / Train Data Num : 251
--------2 / 261---------
Test Data : 2-11 / Train Data Num : 250
--------3 / 261---------
Test Data : 3-12 / Train Data Num : 249
--------4 / 261---------
Test Data : 4-13 / Train Data Num : 248
--------5 / 261---------
Test Data : 5-14 / Train Data Num : 247
--------6 / 261---------
Test Data : 6-15 / Train Data Num : 246
--------7 / 261---------
Test Data : 7-16 / Train Data Num : 245
--------8 / 261---------
Test Data : 8-17 / Train Data Num : 244
--------9 / 261---------
Test Data : 9-18 / Train Data Num : 243
--------10 / 261---------
Test Data : 10-19 / Train Data Num : 242
--------11 / 261---------
Test Data : 11-20 / Train Data Num : 242
--------12 / 261---------
Test Data : 12-21 / Train Data Num : 242
--------13 / 261---------
Test Data : 13-22 / Train Data Num : 242
--------14 / 261---------
Test Data : 14-23 / Train Data Num : 242
--------15 / 261---------
Test Data : 15-24 / Train Data Num : 242
------

--------128 / 261---------
Test Data : 41-50 / Train Data Num : 242
--------129 / 261---------
Test Data : 42-51 / Train Data Num : 242
--------130 / 261---------
Test Data : 43-52 / Train Data Num : 242
--------131 / 261---------
Test Data : 44-53 / Train Data Num : 242
--------132 / 261---------
Test Data : 45-54 / Train Data Num : 242
--------133 / 261---------
Test Data : 46-55 / Train Data Num : 242
--------134 / 261---------
Test Data : 47-56 / Train Data Num : 242
--------135 / 261---------
Test Data : 48-57 / Train Data Num : 242
--------136 / 261---------
Test Data : 49-58 / Train Data Num : 242
--------137 / 261---------
Test Data : 50-59 / Train Data Num : 242
--------138 / 261---------
Test Data : 51-60 / Train Data Num : 242
--------139 / 261---------
Test Data : 52-61 / Train Data Num : 242
--------140 / 261---------
Test Data : 53-62 / Train Data Num : 242
--------141 / 261---------
Test Data : 54-63 / Train Data Num : 242
--------142 / 261---------
Test Data : 55-64 / T

--------257 / 261---------
Test Data : 83-92 / Train Data Num : 247
--------258 / 261---------
Test Data : 84-93 / Train Data Num : 248
--------259 / 261---------
Test Data : 85-94 / Train Data Num : 249
--------260 / 261---------
Test Data : 86-95 / Train Data Num : 250
--------261 / 261---------
Test Data : 87-96 / Train Data Num : 251
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0

In [17]:
    print(voxAc)
    
    # csv書き出し
    PATH_RESULT = PATH + 'ACCURACY[loo]_VOXtimeseries' + str(N) +'_SVM.csv'
    voxAc.to_csv(PATH_RESULT, index = True)

       leave-one-out
Voxel1           0.8
Voxel2           0.4
Voxel3             0
Voxel4           1.5
Voxel5             0
Voxel6             0
Voxel7           0.4
